In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
print(os.listdir("/kaggle/input/"))


In [ ]:
print(os.listdir("/kaggle/input/plantvillage-dataset/"))


In [ ]:
print(os.listdir("/kaggle/input/plantvillage-dataset/color/"))


In [ ]:
import random
# each time the neural networks trains, in randomly generates is parameter values # to avoid different outputs each time

random.seed(0)
import numpy as np
np.random.seed(0)

import tensorflow as tf
tf.random.set_seed(0)
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
import os
import json
from zipfile import ZipFile
from PIL import Image

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

In [ ]:
!ls


In [ ]:
# Correct dataset path
dataset_path = "/kaggle/input/plantvillage-dataset/"

# Print available folders in the dataset
print(os.listdir(dataset_path))

# Print the number of classes and first 5 classes for each dataset version
print(len(os.listdir(dataset_path + "segmented")))
print(os.listdir(dataset_path + "segmented")[:5])

print(len(os.listdir(dataset_path + "color")))
print(os.listdir(dataset_path + "color")[:5])

print(len(os.listdir(dataset_path + "grayscale")))
print(os.listdir(dataset_path + "grayscale")[:5])


In [ ]:
# Correct dataset path
dataset_path = "/kaggle/input/plantvillage-dataset/"

# Check the number of images in the "Grape___healthy" class inside the "color" dataset
grape_healthy_path = dataset_path + "color/Grape___healthy"
print(len(os.listdir(grape_healthy_path)))  # Print total image count

# Print first 5 image filenames
print(os.listdir(grape_healthy_path)[:5])


In [ ]:
base_dir = "/kaggle/input/plantvillage-dataset/color"


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

# Correct dataset path
dataset_path = "/kaggle/input/plantvillage-dataset/color/Apple___Cedar_apple_rust/"

# Pick a random image from the class folder
random_image = os.listdir(dataset_path)[0]  # First image
image_path = os.path.join(dataset_path, random_image)

# Load and display the image
img = mpimg.imread(image_path)

print("Image shape:", img.shape)  # Print image dimensions

plt.imshow(img)
plt.axis('off')  # Hide axes
plt.show()


In [ ]:
import matplotlib.image as mpimg
import os

# Correct dataset path
dataset_path = "/kaggle/input/plantvillage-dataset/color/Apple___Cedar_apple_rust/"

# Pick a random image from the folder
random_image = os.listdir(dataset_path)[0]  # Select first image
image_path = os.path.join(dataset_path, random_image)

# Load image as NumPy array
img = mpimg.imread(image_path)
print("Image shape:", img.shape)  # Print image dimensions
print(img)  # Print pixel values


In [ ]:
# Setting a uniform height and width to all images
img_size = 224
batch_size = 32

In [ ]:
data_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2 # 20% of data for validation
)

In [ ]:
# train generator
train_generator = data_gen.flow_from_directory(
    base_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    subset='training',
    class_mode='categorical'
)

In [ ]:
# validation generator
validation_generator = data_gen.flow_from_directory(
    base_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    subset='validation',
    class_mode='categorical'
)

In [ ]:
from tensorflow.keras import models, layers, optimizers, Input
from tensorflow.keras.callbacks import EarlyStopping

model = models.Sequential([
    Input(shape=(img_size, img_size, 3)),  # ✅ Define input explicitly

    layers.Conv2D(32, (3, 3), padding='same'),
    layers.BatchNormalization(),  # ✅ Move BN before ReLU
    layers.Activation('relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(256, (3, 3), padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(512, (3, 3), padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D((2, 2)),

    layers.GlobalAveragePooling2D(),

    layers.Dense(512),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.4),  # 🔺 Slightly reduce dropout

    layers.Dense(256),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.3),

    layers.Dense(train_generator.num_classes, activation='softmax')  # ✅ Output Layer
])

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.0001),  # Lower learning rate
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
import math

steps_per_epoch = max(1, train_generator.samples // batch_size)
validation_steps = max(1, validation_generator.samples // batch_size)

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    verbose=1
)

In [ ]:
# Check best epoch
# Find the epoch with the best validation accuracy
best_epoch = np.argmax(history.history['val_accuracy']) + 1  # Adding 1 because epoch index starts from 0

print(f"Best epoch: {best_epoch}")
print(f"Best Validation Accuracy: {max(history.history['val_accuracy'])}")


In [51]:
epoch_46_index = 45  # Since epochs are 0-indexed in history

train_accuracy_46 = history.history['accuracy'][epoch_46_index] * 100
val_accuracy_46 = history.history['val_accuracy'][epoch_46_index] * 100

print(f"Train Accuracy at Epoch 46: {train_accuracy_46:.2f}%")
print(f"Validation Accuracy at Epoch 46: {val_accuracy_46:.2f}%")


Train Accuracy at Epoch 46: 0.00%
Validation Accuracy at Epoch 46: 100.00%


In [50]:
best_epoch_loss = np.argmin(history.history['val_loss']) + 1

print(f"Best epoch (lowest loss): {best_epoch_loss}")
print(f"Lowest Validation Loss: {min(history.history['val_loss'])}")


Best epoch (lowest loss): 46
Lowest Validation Loss: 2.7418097943154862e-06


In [49]:
print("Evaluating model...")
val_loss, val_accuracy = model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

Evaluating model...
339/339 ━━━━━━━━━━━━━━━━━━━━ 30s 90ms/step - accuracy: 0.9179 - loss: 0.2769
Validation Accuracy: 92.26%


In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline  

plt.figure(figsize=(8, 5))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.grid()
plt.show()

plt.figure(figsize=(8, 5))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.grid()
plt.show()


In [ ]:
import numpy as np
from PIL import Image

# Function to Load and Preprocess the Image using Pillow
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    try:
        # Load the image and ensure it's in RGB mode
        img = Image.open(image_path).convert('RGB')
        # Resize the image
        img = img.resize(target_size)
        # Convert the image to a numpy array
        img_array = np.array(img)
        # Add batch dimension
        img_array = np.expand_dims(img_array, axis=0)
        # Normalize pixel values to [0, 1]
        img_array = img_array.astype('float32') / 255.0
        return img_array
    except Exception as e:
        print(f"Error loading image: {e}")
        return None

# Function to Predict the Class of an Image
def predict_image_class(model, image_path, train_generator):
    preprocessed_img = load_and_preprocess_image(image_path)
    if preprocessed_img is None:
        return "Error: Unable to process image."

    predictions = model.predict(preprocessed_img)
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    
    # Get class labels (Invert train_generator.class_indices)
    class_indices = {v: k for k, v in train_generator.class_indices.items()}
    predicted_class_name = class_indices.get(predicted_class_index, "Unknown Class")

    return predicted_class_name


In [48]:
# Create a mapping from class indices to class names
class_indices = {v: k for k, v in train_generator.class_indices.items()}

In [47]:
class_indices

{0: 'Apple___Apple_scab',
 1: 'Apple___Black_rot',
 2: 'Apple___Cedar_apple_rust',
 3: 'Apple___healthy',
 4: 'Blueberry___healthy',
 5: 'Cherry_(including_sour)___Powdery_mildew',
 6: 'Cherry_(including_sour)___healthy',
 7: 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 8: 'Corn_(maize)___Common_rust_',
 9: 'Corn_(maize)___Northern_Leaf_Blight',
 10: 'Corn_(maize)___healthy',
 11: 'Grape___Black_rot',
 12: 'Grape___Esca_(Black_Measles)',
 13: 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 14: 'Grape___healthy',
 15: 'Orange___Haunglongbing_(Citrus_greening)',
 16: 'Peach___Bacterial_spot',
 17: 'Peach___healthy',
 18: 'Pepper,_bell___Bacterial_spot',
 19: 'Pepper,_bell___healthy',
 20: 'Potato___Early_blight',
 21: 'Potato___Late_blight',
 22: 'Potato___healthy',
 23: 'Raspberry___healthy',
 24: 'Soybean___healthy',
 25: 'Squash___Powdery_mildew',
 26: 'Strawberry___Leaf_scorch',
 27: 'Strawberry___healthy',
 28: 'Tomato___Bacterial_spot',
 29: 'Tomato___Early_blight',
 30: '

In [ ]:
# saving the class names as json file
json.dump(class_indices, open('class_indices.json', 'w'))

In [54]:
model.save("my_trained_model.h5")  # Saving the entire model


In [55]:
from tensorflow.keras.models import load_model
model = load_model("my_trained_model.h5")

# On reopening Kaggle I just need to run this cell instead of retraining

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!mv my_model.h5 /content/drive/MyDrive/